# Barrier Option

In [1]:
import QuantLib as ql 

In [2]:
today = ql.Date(25, ql.March, 2022)
ql.Settings.instance().evaluationDate = today

In [3]:
print(ql.Settings.instance().evaluationDate)

March 25th, 2022


In [4]:
# knock-in barrier option

exp_date = today + ql.Period(3, ql.Months)

option = ql.BarrierOption(ql.Barrier.UpIn,
                          120, # barrier
                          0, # rebate
                          ql.PlainVanillaPayoff(ql.Option.Call, 100),
                          ql.EuropeanExercise(exp_date))

In [5]:
# constant values

u = ql.SimpleQuote(100)
r = ql.SimpleQuote(0.01)
sigma = ql.SimpleQuote(0.2)

In [6]:
# interest rate curve
# volatility curve

risk_free_curve = ql.FlatForward(0, ql.TARGET(), ql.QuoteHandle(r), ql.Actual360())
volatility = ql.BlackConstantVol(0, ql.TARGET(), ql.QuoteHandle(sigma), ql.Actual360())

In [7]:
# get process

process = ql.BlackScholesProcess(ql.QuoteHandle(u),
                                 ql.YieldTermStructureHandle(risk_free_curve),
                                 ql.BlackVolTermStructureHandle(volatility))

In [8]:
# get and set engine

engine = ql.AnalyticBarrierEngine(process)
option.setPricingEngine(engine)

In [9]:
print(f"Option value: {option.NPV():.4f}")

Option value: 1.3658


**engine doesn't provide the greeks**

In [10]:
option.delta()

RuntimeError: delta not provided

In [11]:
option.gamma()

RuntimeError: gamma not provided

In [12]:
option.vega()

RuntimeError: vega not provided

## Numerical calculation

$$\Delta = \frac{P(u_0+h) - P(u_0-h)}{2h}$$ <br><br>
$$\Gamma = \frac{P(u_0+h) - 2P(u_0) + P(u_0-h)}{h^2}$$

In [13]:
u0 = u.value()
h = 0.01
P0 = option.NPV()

In [14]:
u.setValue(u0 + h)
P_plus = option.NPV()

u.setValue(u0 - h)
P_minus = option.NPV()

u.setValue(u0)

In [15]:
print(P0, P_plus, P_minus)

1.3657980739109867 1.3688112201958083 1.3627900998610207


In [16]:
delta = (P_plus - P_minus) / (2*h)
gamma = (P_plus - 2*P0 + P_minus) / (h**2)

In [17]:
print(f"Delta: {delta:.4f}")
print(f"Gamma: {gamma:.4f}")

Delta: 0.3011
Gamma: 0.0517


$$\frac{\partial P}{\partial x} = \frac{P(x_0+h) - P(x_0)}{h}$$

In [18]:
# Rho --> change in risk-free rate

r0 = r.value()
h = 0.0001

r.setValue(r0 + h)
P_plus = option.NPV()

r.setValue(r0)

rho = (P_plus - P0) / h
print(f"Rho: {rho:.4f}")

Rho: 6.5310


In [19]:
# Vega --> change in volatility

sigma0 = sigma.value()
h = 0.0001

sigma.setValue(sigma0 + h)
P_plus = option.NPV()

sigma.setValue(sigma0)

vega = (P_plus - P0) / h
print(f"Vega: {vega:.4f}")

Vega: 26.5252


In [20]:
# Theta --> change in time

ql.Settings.instance().evaluationDate = today + 1
P1 = option.NPV()
h = 1/365

ql.Settings.instance().evaluationDate = today

theta = (P1 - P0) / h
print(f"Theta: {theta:.4f}")

Theta: -32.2541
